In [ ]:
import SimpleITK as sitk
import numpy
import os 
import glob
import re
import matplotlib.pyplot as plt

%matplotlib inline

#import scipy.ndimage
#from parameters import *

In [ ]:
locationImages = "/Volumes/Gatti_Files/Users/Gatti/Documents/Masters/T2_Study/MRI_Data_Biomech_Images_Official/MRI/Analyzed/exam_folder/QMetrics"

In [ ]:
def readImage(pathDicom):
    reader = sitk.ImageSeriesReader()
    filenamesDICOM = reader.GetGDCMSeriesFileNames(pathDicom)
    reader.SetFileNames(filenamesDICOM)
    imgOriginal = reader.Execute()
    return(imgOriginal)

The below section imports all of the dicom T2 images. 

In [ ]:
dictionaryT2Images = {}
os.chdir(locationImages)
examFolders = glob.glob('exam_*')
for exam in examFolders:
    os.chdir(exam)
    t2Series = glob.glob('Ser*00')
    print(t2Series)
    examNumber = str(re.sub('exam_', '', exam))
    seriesName = str(re.sub('00', '', t2Series[0]))
    seriesNumber = str(re.sub('Ser', '', seriesName))
    dictionaryT2Images[exam] = readImage(seriesName + '/')
    os.chdir('..')

Extract just the first echo for each slice and create a new sitk object of 3d with just that number of slices. 

In [ ]:
firstEcho = {}
echos = 8
inplaneResolution = (256,256)
for image in dictionaryT2Images:
    image2PlayWith = dictionaryT2Images[image]
    totalSlices = image2PlayWith.GetSize()[2]
    actualSlices = totalSlices/echos
    numpyImage = sitk.GetArrayFromImage(image2PlayWith)
    outputImage = numpyImage[0:totalSlices:echos, :, :]
    firstEcho[image] = sitk.GetImageFromArray(outputImage)

Extract a crude area that includes the patella. This is much larger than before. This will hopefully facilitate registration because we should have some femur and maybe tibia in the image as well. 

In [ ]:
area2Register = {}
iDimension = 128
jDimension = 154
for image in firstEcho:
    image2Process = firstEcho[image]
    numpyImage = sitk.GetArrayFromImage(image2Process)
    slices = image2Process.GetSize()[2]
    outputImage = numpy.zeros([26, jDimension, iDimension])
    outputImage[0:slices, :,:] = numpyImage[:, 0:jDimension, 0:iDimension]
    area2Register[image] = sitk.GetImageFromArray(outputImage)   
    

In [ ]:
testArray = sitk.GetArrayFromImage(area2Register[image])
# for i in range(26):
#     plt.figure()
#     plt.imshow(testArray[i,:,:])
plt.imshow(testArray[12,:,:])

Create a 4D vector of all the images that we are going to register to one another. 

In [ ]:
vectorOfImages = sitk.VectorOfImage()
for image in area2Register:
    insertImage = sitk.Cast(area2Register[image], sitk.sitkInt16)
    vectorOfImages.push_back(insertImage)

vectorImage2Register = sitk.JoinSeries(vectorOfImages)

In [ ]:
elastix = sitk.SimpleElastix()
elastix.SetFixedImage(vectorImage2Register)
elastix.SetMovingImage(vectorImage2Register)
elastix.SetParameterMap(sitk.GetDefaultParameterMap('groupwise'))
elastix.Execute()

In [ ]:
sitk.GetDefaultParameterMap("translation"))

In [ ]:
groupwiseRegistration = elastix.GetResultImage()
groupwiseSlices = groupwiseRegistration.GetSize()[2]
groupwiseImages = groupwiseRegistration.GetSize()[3]
groupwiseInPlane = groupwiseRegistration.GetSize()[0:2]
print('Common number of slices: ' + str(groupwiseSlices))
print('Number of image sets present: ' + str(groupwiseImages))
print('In-plane image resolution: ' + str(groupwiseInPlane))